In [ ]:
!git clone https://github.com/fe1ixxu/ALMA.git
!cd ALMA
!pip install -r requirements.txt


fatal: destination path 'ALMA' already exists and is not an empty directory.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [1]:
pip install peft

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

class ALMATranslator:
    def __init__(self, model_name: str):
        self.model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="cuda")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

    def translate(self, text: str, src_lang: str, tgt_lang: str) -> str:
        prompt = f"Translate this from {src_lang} to {tgt_lang}:\n{src_lang}: {text}\n{tgt_lang}:"
        input_ids = self.tokenizer(prompt, return_tensors="pt", padding=True, max_length=256, truncation=True).input_ids.cuda()

        with torch.no_grad():
            generated_ids = self.model.generate(input_ids=input_ids, num_beams=5, max_new_tokens=50, do_sample=True, temperature=0.6, top_p=0.9)

        translated_text = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return translated_text.split(f"{tgt_lang}:")[1].strip()




In [ ]:
# if __name__ == "__main__":
model_name = "haoranxu/ALMA-7B-R"

# Tradutor de Português para Inglês
pt_to_en_translator = ALMATranslator(model_name=model_name)

# Tradutor de Inglês para Português
# en_to_pt_translator = ALMATranslator(model_name=model_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM
from transformers import LlamaTokenizer

# Load base model and LoRA weights
model = AutoModelForCausalLM.from_pretrained("haoranxu/ALMA-7B-Pretrain", torch_dtype=torch.float16, device_map="auto")
model = PeftModel.from_pretrained(model, "haoranxu/ALMA-7B-Pretrain-LoRA")
tokenizer = LlamaTokenizer.from_pretrained("haoranxu/ALMA-7B-Pretrain", padding_side='left')



pytorch_model-00002-of-00003.bin:  42%|####2     | 4.17G/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

In [8]:
# Add the source setence into the prompt template
prompt="Translate this from Portuguese to English:\n Portuguese: Meu presidente! \nEnglish:"
input_ids = tokenizer(prompt, return_tensors="pt", padding=True, max_length=40, truncation=True).input_ids.cuda()

# Translation
with torch.no_grad():
    generated_ids = model.generate(input_ids=input_ids, num_beams=5, max_new_tokens=200, do_sample=True, temperature=0.6, top_p=0.9)
outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
print(outputs)

['Translate this from Portuguese to English:\n Portuguese: Meu presidente! \nEnglish:My president!']
